# Chebyshev polynomials and fitting workflows

Anton Antonov  
June 2024   
December 2024

-----

## Introduction

Let us list the full set of features and corresponding packages:

- ["JavaScript::Google::Charts"](https://raku.land/zef:antononcube/JavaScript::Google::Charts)
    - Scatter plots
    - Time series data visualization
- ["Math::Polynomial::Chebyshev"](https://raku.land/zef:antononcube/Math::Polynomial::Chebyshev)
    - Polynomial basis
    - Both recursive and trigonometric methods of computation
    - The recursive method provides exact (bignum) integers for the numerators and denominators
- ["Math::Fitting"](https://raku.land/zef:antononcube/Math::Fitting)
    - Linear regression (i.e. fitting) with function bases
    - Gives functors as results
    - Multiple properties of the functors can be retrieved

- ["Data::TypeSystem"](https://raku.land/zef:antononcube/Data::TypeSystem)
    - Summary of data types

- ["Data::Summarizers"](https://raku.land/zef:antononcube/Data::Summarizers)
    - Summary of data values

### TL;DR

- Chebyshev polynomials can be exactly computed
- The "Math::Fitting" package produces functors
- The fitting is done with a function basis
- Matrix formulas are used to compute the fit (linear regression)
- Real life example is shown with weather temperature data 
    - You can just see the section before the last.

-----

## Setup

In [40]:
use Math::Matrix;
use Math::Polynomial::Chebyshev;
use Math::Fitting;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::Importers;

use JavaScript::D3;
use JavaScript::Google::Charts;

use Hash::Merge;
use LLM::Configurations;

### Google Charts

In [41]:
#% javascript
google.charts.load('current', {'packages':['corechart']});
google.charts.load('current', {'packages':['gauge']});
google.charts.load('current', {'packages':['wordtree']});
google.charts.load('current', {'packages':['geochart']});
google.charts.load('current', {'packages':['table']});
google.charts.load('current', {'packages':['line']});
google.charts.setOnLoadCallback(function() {
    console.log('Google Charts library loaded');
});


#### Dark mode

In [42]:
my $format = 'html';
my $titleTextStyle = { color => 'Ivory', fontSize => 16 };
my $backgroundColor = '#1F1F1F';
my $legendTextStyle = { color => 'Silver' };
my $legend = { position => "none", textStyle => {fontSize => 14, color => 'Silver'} };

my $hAxis = { title => 'x', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};
my $vAxis = { title => 'y', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};

my $annotations = {textStyle => {color => 'Silver', fontSize => 10}};
my $chartArea = {left => 50, right => 50, top => 50, bottom => 50, width => '90%', height => '90%'};

{bottom => 50, height => 90%, left => 50, right => 50, top => 50, width => 90%}

#### Light mode

In [43]:
#`[
my $format = 'html';
my $titleTextStyle = { color => 'DimGray', fontSize => 16 };
my $backgroundColor = 'White';
my $legendTextStyle = { color => 'DarkGray' };
my $legend = { position => "none", textStyle => {fontSize => 14, color => 'DarkGray'} };

my $hAxis = { title => 'x', titleTextStyle => { color => 'DimGray' }, textStyle => { color => 'DarkGray'}, logScale => False, format => 'decimal'};
my $vAxis = { title => 'y', titleTextStyle => { color => 'DimGray' }, textStyle => { color => 'DarkGray'}, logScale => False, format => 'decimal'};

my $annotations = {textStyle => {color => 'DarkGray', fontSize => 10}};
my $chartArea = {left => 50, right => 50, top => 50, bottom => 50, width => '90%', height => '90%'};
]

()

------

## Why use Chebyshev polynomials in fitting?

[Chebyshev polynomials](https://en.wikipedia.org/wiki/Chebyshev_polynomials) provide a powerful and efficient basis for linear regression fitting, particularly when dealing with polynomial approximation and curve fitting. These polynomials, defined recursively, are a sequence of orthogonal polynomials that minimize the problem of [Runge's phenomenon](https://en.wikipedia.org/wiki/Runge's_phenomenon), which is common with high-degree polynomial interpolation.

One of the key advantages of using Chebyshev polynomials in regression is their property of minimizing the maximum error between the fitted curve and the actual data points, known as the _minimax property_. Because of that property, more stable and accurate approximations are obtained, especially at the boundaries of the interval.

The orthogonality of Chebyshev polynomials with respect to the weight function $w(x) = \frac{1}{\sqrt{1-x^2}}$ on the interval $[-1, 1]$ ensures that the regression coefficients are uncorrelated, which simplifies the computation and enhances numerical stability. Furthermore, Chebyshev polynomials are excellent for approximating functions that are not well-behaved or have rapid oscillations, as they distribute approximation error more evenly across the interval.

**Remark:** This is one of the reasons [Clenshaw-Curtis quadrature](https://en.wikipedia.org/wiki/Clenshaw–Curtis_quadrature) was one of "main" quadrature rules I implemented in [Mathematica's `NIntegerate`](https://reference.wolfram.com/language/tutorial/NIntegrateIntegrationRules.html#486402291).

Using Chebyshev polynomials into linear regression models allows for a flexible and robust function basis that can adapt to the complexity of the data while maintaining computational efficiency. This makes them particularly suitable for applications requiring high precision and stability, such as in signal processing, numerical analysis, and scientific computing. 

Overall, the unique properties of Chebyshev polynomials make them a great regression tool, offering a blend of accuracy, stability, and efficiency.

-------

## Chebyshev polynomials computation

This section discusses the computation of Chebyshev polynomials using different methods and their implications on precision.


### Computation granularity

The computation over Chebyshev polynomials is supported on the interval $[-1, 1]$. The recursive and trigonometric methods are compared to understand their impact on the precision of results.

In [44]:
<recursive trigonometric>
==> { .map({ $_ => chebyshev-t(3, 0.3, method => $_) }) }()

(recursive => -0.792 trigonometric => -0.7920000000000003)

Here we compute polynomial values over a "dense enough" grid:

In [45]:
my $k = 12;

# Whatever goes to 'recursive'
my $method = 'trig'; # 'trig'

my @x = (-1.0, -0.99 ... 1.0);
say '@x.elems : ', @x.elems;

my @data  = @x.map({ [$_, chebyshev-t($k, $_, :$method)]});
my @data1 = chebyshev-t($k, @x);

say deduce-type(@data);
say deduce-type(@data1);

@x.elems : 201
Vector(Tuple([Atom((Rat)), Atom((Numeric))]), 201)
Vector((Any), 201)


Let use the residuals with trigonometric and recursive:

In [46]:
sink records-summary(@data.map(*.tail) <<->> @data1)

+----------------------------------+
| numerical                        |
+----------------------------------+
| Min    => -3.774758283725532e-15 |
| Median => -3.469446951953614e-18 |
| 1st-Qu => -6.661338147750939e-16 |
| Mean   => -8.803937402208662e-17 |
| Max    => 3.4416913763379853e-15 |
| 3rd-Qu => 3.3306690738754696e-16 |
+----------------------------------+


### Precision

We can compute the exact Chebyshev polynomial values at given points using `FatRat` numbers:

In [47]:
my $v = chebyshev-t(100, <1/4>.FatRat, method => 'recursive')

0.9908630290911637341902191815340830456

Here are the numerator and denominator:

In [48]:
say $v.numerator;
say $v.denominator;

2512136227142750476878317151377
2535301200456458802993406410752


-----

## Plots

This section shows how the plot the Chebyshev polynomials using [Google Charts](https://developers.google.com/chart) via ["JavaScript::Google::Charts"](https://raku.land/zef:antononcube/JavaScript::Google::Charts).

### Single polynomial

Single polynomial plot using a [Line chart](https://developers.google.com/chart/interactive/docs/gallery/linechart):

In [49]:
#%html
my $n = 6;
my @data = chebyshev-t(6, (-1, -0.98 ... 1).List);
js-google-charts('LineChart', @data, 
    title => "Chebyshev-T($n) polynomial", 
    :$titleTextStyle, :$backgroundColor, :$chartArea, :$hAxis, :$vAxis,
    width => 800, 
    div-id => 'poly1', :$format,
    :png-button)

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

### Basis

Doing fitting we are interested in using bases of functions. Here for first eight Chebyshev-T polynomials make plot data:

In [50]:
my $n = 8;
my @data = (-1, -0.98 ... 1).map(-> $x { [x => $x, |(0..$n).map({ $_.Str => chebyshev-t($_, $x, :$method) }) ].Hash });

deduce-type(@data):tally;

Tuple([Struct([0, 1, 2, 3, 4, 5, 6, 7, 8, x], [Num, Num, Num, Num, Num, Num, Num, Num, Num, Int]) => 1, Struct([0, 1, 2, 3, 4, 5, 6, 7, 8, x], [Num, Num, Num, Num, Num, Num, Num, Num, Num, Rat]) => 100], 101)

Here is the plot with all eight functions:

In [51]:
#%html
js-google-charts('LineChart', @data,
    column-names => ['x', |(0..$n)».Str],
    title => "Chebyshev T polynomials, 0 .. $n",
    :$titleTextStyle,
    width => 800, 
    height => 400,
    :$backgroundColor, :$hAxis, :$vAxis,
    legend => merge-hash($legend, %(position => 'right')),
    chartArea => merge-hash($chartArea, %(right => 100)),
    format => 'html', 
    div-id => "cheb$n",
    :$format,
    :png-button)

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

-----

## Text plot

*Text plots always work!*

In order to plot with ["Text::Plot"](https://raku.land/zef:antononcube/Text::Plot) 
the data has to be converted into [long form](https://en.wikipedia.org/wiki/Wide_and_narrow_data) first:

In [52]:
my @dataLong = to-long-format(@data, <x>).sort(*<Variable x>);
deduce-type(@dataLong):tally

Tuple([Struct([Value, Variable, x], [Num, Str, Int]) => 9, Struct([Value, Variable, x], [Num, Str, Rat]) => 900], 909)

Here is a sample:

In [53]:
#% html
@dataLong.pick(10)
==> {.sort(*<Variable x>)}()
==> to-html(field-names => <Variable x Value>)

Variable,x,Value
2,-0.18,-0.9352
4,0.72,-0.99729152
5,-0.06,-0.29569244159999925
6,-0.68,0.22389896396800207
6,-0.38,0.6946846842879992
6,0.2,-0.3547519999999992
6,0.52,0.9902838292480001
6,0.66,0.37785312051200004
7,0.62,0.99951362957312
8,-0.84,-0.12396443184005182


Here is the text plot:

In [54]:
my @chebInds = 1, 2, 3, 4;
my @dataLong3 = @dataLong.grep({ $_<Variable>.Int ∈ @chebInds }).classify(*<Variable>).map({ $_.key => $_.value.map(*<x Value>).Array }).sort(*.key)».value;
say @chebInds Z=> <* □ ▽ ❍>; 
text-list-plot(@dataLong3, width => 100, height => 25, title => "Chebyshev T polynomials, 0 .. $n", x-label => (@chebInds >>~>> ' : ' Z~ <* □ ▽ ❍>).join(', '))

(1 => * 2 => □ 3 => ▽ 4 => ❍)


                                  Chebyshev T polynomials, 0 .. 8                                   
+----+---------------------+---------------------+---------------------+---------------------+-----+      
|                                                                                                  |      
+    ❍                  ▽▽▽▽▽▽▽▽               ❍❍❍❍❍❍                                       *❍     +  1.00
|     □              ▽▽▽        ▽▽▽         ❍❍❍      ❍❍❍                               ***** □     |      
|      □□          ▽▽              ▽▽     ❍❍            ❍                          ****    □□▽     |      
|     ❍  □        ▽▽                 ▽▽▽ ❍               ❍❍                   *****       □ ▽❍     |      
|         □      ▽                     ❍❍▽                 ❍❍             *****          □         |      
+          □□   ▽                     ❍  ▽▽                  ❍        ****             □□  ▽       +  0.50
|      ❍    □  ▽                     ❍     

-----

## Fitting

Here we generate "measurements data" with noise:

In [55]:
my @temptimelist = 0.1, 0.2 ... 20;
my @tempvaluelist = @temptimelist.map({ sin($_) / $_ }) Z+ (1..200).map({ (3.rand - 1.5) * 0.02 });
my @data1 = @temptimelist Z @tempvaluelist;
@data1 = @data1.deepmap({ .Num });

deduce-type(@data1)

Vector(Vector(Atom((Numeric)), 2), 200)

Rescaling of the x-coordinates:

In [56]:
my @data2 = @data1.map({ my @a = $_.clone; @a[0] = @a[0] / max(@temptimelist); @a });

deduce-type(@data2)

Vector(Vector(Atom((Numeric)), 2), 200)

Here is a summary:

In [57]:
sink records-summary(@data2)

+------------------+--------------------------------+
| 0                | 1                              |
+------------------+--------------------------------+
| Min    => 0.005  | Min    => -0.23726746909509286 |
| 1st-Qu => 0.2525 | 1st-Qu => -0.05996221115344534 |
| Mean   => 0.5025 | Mean   => 0.07421865809764845  |
| Median => 0.5025 | Median => 0.010618520236770707 |
| 3rd-Qu => 0.7525 | 3rd-Qu => 0.0854017888724484   |
| Max    => 1      | Max    => 1.0221779525812003   |
+------------------+--------------------------------+


Here is a plot of that data:

In [58]:
#% html
js-google-charts("Scatter", @data2, 
    title => 'Measurements data with noise',
    :$backgroundColor, :$hAxis, :$vAxis,
    :$titleTextStyle, :$chartArea,
    width => 800, 
    div-id => 'data', :$format,
    :png-button)

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

Make a function that rescales from $[0, 1]$ to $[-1, 1]$:

In [59]:
my &rescale = { ($_ - 0.5) * 2 };

-> ;; $_? is raw = OUTER::<$_> { #`(Block|5164183324600) ... }

Here is a list of basis functions:

In [60]:
my @basis = (^16).map({ chebyshev-t($_) o &rescale });
@basis.elems

16

**Remark:** Function composition operator `o` is used above. Before computing the Chebyshev polynomial value the argument is rescaled.

Here we compute a linear model fit with those functions:

In [61]:
my &lm = linear-model-fit(@data2, :@basis)

Math::Fitting::FittedModel(type => linear, data => (200, 2), response-index => 1, basis => 16)

Here are the best fit parameters:

In [62]:
&lm('BestFitParameters')

[0.18397175612508504 -0.3424086829657002 0.30185404252949427 -0.20548253491856996 0.12476469252163935 -0.0036110533303903925 -0.04202409602318209 0.08350980619157693 -0.05956643516148638 -0.04110582976324682 0.04315573560408151 0.004128829412964724 -0.0068140601108521 -0.004498256798412357 0.0028544659910417344 -0.0056255030668945945]

Here is a plot of those parameters:

In [63]:
#% html
js-google-charts("Bar", &lm('BestFitParameters'), 
    :!horizontal,
    title => 'Best fit parameters',
    :$backgroundColor, 
    hAxis => merge-hash($hAxis, {title => 'Basis function index'}), 
    vAxis => merge-hash($hAxis, {title => 'Coefficient'}), 
    :$titleTextStyle, :$chartArea,
    width => 800, 
    div-id => 'bestFitParams', :$format,
    :png-button)

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

We can see from the plot that using more the 12 basis functions for that data is not improving the fit, since the coefficients after the 12th index are very small.

Now, let us plot the data and the fit. First we prepare the plot data:

In [64]:
my @fit = @data2.map(*.head)».&lm;
my @plotData = transpose([@data2.map(*.head).Array, @data2.map(*.tail).Array, @fit]);
@plotData = @plotData.map({ <x data fit>.Array Z=> $_.Array })».Hash;

deduce-type(@plotData)

Vector(Assoc(Atom((Str)), Atom((Numeric)), 3), 200)

Here is the plot:

In [65]:
#% html
js-google-charts('ComboChart', 
    @plotData, 
    title => 'Data and fit',
    column-names => <x data fit>,
    :$backgroundColor, :$titleTextStyle :$hAxis, :$vAxis,
    seriesType => 'scatter',
    series => {
        0 => {type => 'scatter', pointSize => 5, opacity => 0.1, color => 'DarkGray'},
        1 => {type => 'line', lineWidth => 3}
    },
    legend => merge-hash($legend, %(position => 'bottom')),
    :$chartArea,
    width => 800, 
    div-id => 'fit1', :$format,
    :png-button)

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

Compute the residuals of the last fit:

In [66]:
sink records-summary( (@fit <<->> @data2.map(*.tail))».abs )

+----------------------------------+
| numerical                        |
+----------------------------------+
| Min    => 5.2097167823283996e-05 |
| Median => 0.011890325384604682   |
| Mean   => 0.013186788112861312   |
| 3rd-Qu => 0.020535930325408798   |
| Max    => 0.03419150796023318    |
| 1st-Qu => 0.005867961809887827   |
+----------------------------------+


----

## Condition number

The formula with which the [Ordinary Least Squares (OLS)](https://en.wikipedia.org/wiki/Ordinary_least_squares) fit is computed is:

$$
\beta = (X^T \cdot X)^{-1} \cdot X^T \cdot y
$$

Let us look into the condition number of the "normal matrix" (or "Gram matrix") $X^T \cdot X$ . First, we get the design matrix:

In [67]:
my @a = &lm.design-matrix();
my $X = Math::Matrix.new(@a);
$X.size

(200 16)

Here is the Gram matrix:

In [68]:
my $g = $X.transposed.dot-product($X);
$g.size

(16 16)

And here is the [condition number](https://en.wikipedia.org/wiki/Condition_number) of that matrix:

In [69]:
$g.condition

88.55110861577737

We conclude that we are fine to use that design matrix.

**Remark:** For a system of linear equations in matrix form $A x = b$, the condition number of $A$, $\kappa (A)$, is defined to be the maximum ratio of the relative error in $x$ to the relative error in $b$.

**Remark:** Typically, if the condition number is $\kappa (A)=10^{d}$, we can expect to lose as many as $d$ digits of accuracy 
in addition to any loss caused by the numerical method (due to precision issues in arithmetic calculations.)

**Remark:** A very "Raku-way" to define ill-conditioned matrix as "almost is not of full rank," or "if its inverse does not exist."
 

-----

## Temperature data

Let us redo the whole workflow with a real life data -- weather temperature data for 4 consecutive years of Greenville, South Carolina, USA. 
(Where the [Perl and Raku Conference 2025](https://www.perl.com/article/get-ready-for-the-2025-perl-and-raku-conference/) is going to be held.)

Here we ingest the time series data:

In [70]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/dsTemperature-Greenville-SC-USA.csv';
my @dsTemperature = data-import($url, headers => 'auto');
@dsTemperature = @dsTemperature.deepmap({ $_ ~~ / ^ \d+ '-' / ?? DateTime.new($_) !! $_.Num });
deduce-type(@dsTemperature)

Vector(Struct([AbsoluteTime, Date, Temperature], [Num, DateTime, Num]), 1462)

Show data summary:

In [71]:
sink records-summary(@dsTemperature, field-names => <Date AbsoluteTime Temperature>)

+--------------------------------+----------------------+------------------------------+
| Date                           | AbsoluteTime         | Temperature                  |
+--------------------------------+----------------------+------------------------------+
| Min    => 2018-01-01T00:00:37Z | Min    => 3723753600 | Min    => -5.72              |
| 1st-Qu => 2019-01-01T00:00:37Z | 1st-Qu => 3755289600 | 1st-Qu => 10.5               |
| Mean   => 2020-01-01T12:00:37Z | Mean   => 3786868800 | Mean   => 17.053549931600518 |
| Median => 2020-01-01T12:00:37Z | Median => 3786868800 | Median => 17.94              |
| 3rd-Qu => 2021-01-01T00:00:37Z | 3rd-Qu => 3818448000 | 3rd-Qu => 24.11              |
| Max    => 2022-01-01T00:00:37Z | Max    => 3849984000 | Max    => 29.89              |
+--------------------------------+----------------------+------------------------------+


Here is a plot:

In [72]:
#% html
js-google-charts("Scatter", @dsTemperature.map(*<Date Temperature>), 
    title => 'Temperature of Greenville, SC, USA',
    :$backgroundColor,
    hAxis => merge-hash($hAxis, {title => 'Time', format => 'M/yy'}), 
    vAxis => merge-hash($hAxis, {title => 'Temperature, ℃'}), 
    :$titleTextStyle, :$chartArea,
    width => 1200, 
    height => 400, 
    div-id => 'tempData', :$format,
    :png-button)

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

Here is a fit -- note the rescaling:

In [73]:
my ($min, $max) = @dsTemperature.map(*<AbsoluteTime>).Array.&{ (.min, .max) }();

(3723753600 3849984000)

In [74]:
my &rescale-time = { -($max + $min) / ($max - $min) + (2 * $_) / ($max - $min)};
my @basis = (^16).map({ chebyshev-t($_) o &rescale-time });
@basis.elems

16

In [75]:
my &lm-temp = linear-model-fit(@dsTemperature.map(*<AbsoluteTime Temperature>), :@basis)

Math::Fitting::FittedModel(type => linear, data => (1462, 2), response-index => 1, basis => 16)

Her is a plot of the time series and the fit:

In [76]:
my @fit = @dsTemperature.map(*<AbsoluteTime>)».&lm-temp;
my @plotData = transpose([@dsTemperature.map({ $_<AbsoluteTime> }).Array, @dsTemperature.map(*<Temperature>).Array, @fit]);
@plotData = @plotData.map({ <x data fit>.Array Z=> $_.Array })».Hash;

deduce-type(@plotData)

Vector(Assoc(Atom((Str)), Atom((Numeric)), 3), 1462)

In [77]:
#% html

my @ticks = @dsTemperature.map({ %( v => $_<AbsoluteTime>, f => $_<Date>.Str.substr(^7)) })».Hash[0, 120 ... *];

js-google-charts('ComboChart', 
    @plotData,
    title => 'Temperature data and Least Squares fit',
    column-names => <x data fit>,
    :$backgroundColor, :$titleTextStyle,
    hAxis => merge-hash($hAxis, {title => 'Time', :@ticks, textPosition => 'in'}), 
    vAxis => merge-hash($hAxis, {title => 'Temperature, ℃'}), 
    seriesType => 'scatter',
    series => {
        0 => {type => 'scatter', pointSize => 3, opacity => 0.1, color => 'Gray'},
        1 => {type => 'line', lineWidth => 4}
    },
    legend => merge-hash($legend, %(position => 'bottom')),
    :$chartArea,
    width => 1200, 
    height => 400, 
    div-id => 'tempDataFit', :$format,
    :png-button)

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

-----

## Future plans

At this point it should be clear that Raku is fully equipped to do regression analysis for both didactical and "real-life" purposes.

I plan to implement in Raku next year the necessary computational frameworks to do [Quantile Regression](https://en.wikipedia.org/wiki/Quantile_regression).

The workflow code in this post can be generated using LLMs -- I plan to write about that soon.